In [1]:
import numpy as np 
import pandas as pd 

import os
import re
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/zemberekwords/stop-words.tr.txt
/kaggle/input/fasttextturkishvector/cc.tr.300.vec
/kaggle/input/dogus-teknoloji-n11/train_n11.csv
/kaggle/input/dogus-teknoloji-n11/test_n11.csv
/kaggle/input/dogus-teknoloji-n11/sample_submission_n11.csv


In [2]:
sample_submission = pd.read_csv('../input/dogus-teknoloji-n11/sample_submission_n11.csv')
test_data = pd.read_csv('../input/dogus-teknoloji-n11/test_n11.csv',sep='|')
train_data = pd.read_csv('../input/dogus-teknoloji-n11/train_n11.csv',sep='|')

In [3]:
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from nltk.tokenize import word_tokenize
import fasttext

In [4]:
with open('../input/zemberekwords/stop-words.tr.txt', 'r') as f:
    stop_words = [line.strip() for line in f]
    missing = ('sayede','mükemmel',)
    stop_words.extend(missing)
    

In [5]:
def toClean(text):

    text = re.sub(r"I", "ı", text)
    text = text.lower()
    text = re.sub('ürün detayları', '', text)
    text = BeautifulSoup(text, "lxml").text
    text = re.sub(r"\n", "  ", text)
    text = re.sub(r"  ", " ", text)
    text = re.sub(' +', ' ', text)
    text = text.strip()
    text = re.sub(r'[^\w\s]',' ',text)
    return text

def cleanStopWords(text):
    words = word_tokenize(text)
    filtered_sentence = " ".join(w.lower() for w in words if not w in stop_words)
    return filtered_sentence


In [6]:
train_data["CATEGORY_ID"] = train_data["CATEGORY_ID"].apply(lambda x: "__label__" + str(x))


train_data['clean_des'] = train_data['DESCRIPTION'].apply(toClean).apply(cleanStopWords)
train_data['clean_title'] = train_data['TITLE'].apply(toClean).apply(cleanStopWords)
train_data['sumOfBag'] = train_data['clean_des'] + ' ' + train_data['clean_title'] 
df = train_data['sumOfBag']

#train_data.CATEGORY_ID.value_counts().to_frame().head()
#y = train_data['CATEGORY_ID']

/opt/conda/lib/python3.7/site-packages/bs4/__init__.py:343: MarkupResemblesLocatorWarning: "." looks like a directory name, not markup. You may want to open a file found in this directory and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning


In [7]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tr.300.vec.gz
!gzip -d cc.tr.300.vec.gz

--2021-10-28 18:57:25--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tr.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1261500728 (1.2G) [binary/octet-stream]
Saving to: ‘cc.tr.300.vec.gz’

cc.tr.300.vec.gz    100%[===================>]   1.17G  45.6MB/s    in 24s     

2021-10-28 18:57:49 (50.1 MB/s) - ‘cc.tr.300.vec.gz’ saved [1261500728/1261500728]



In [8]:
import csv
train_data[['CATEGORY_ID', 'sumOfBag']].to_csv('train.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")

In [9]:
model = fasttext.train_supervised(
    input="train.txt", 
    epoch=150, 
    dim=300,
    wordNgrams=2,
    pretrainedVectors="cc.tr.300.vec"
)

Read 14M words
Number of words:  190905
Number of labels: 177
Progress: 100.0% words/sec/thread:  383197 lr:  0.000000 avg.loss:  0.055366 ETA:   0h 0m 0s


In [10]:
test_data['clean_des'] = test_data['DESCRIPTION'].apply(toClean).apply(cleanStopWords)
test_data['clean_title'] = test_data['TITLE'].apply(toClean).apply(cleanStopWords)
test_data['sumOfBag'] = test_data['clean_des'] + ' . ' + test_data['clean_title'] 
df2 = test_data['sumOfBag']

In [11]:
# Evaluating performance on the entire test file
pred = []
for i in test_data['sumOfBag']:
    pred.append(re.sub('__label__', '', model.predict(i)[0][0]))

In [12]:
test_id = test_data['ID']
expected_df = pd.DataFrame(pred)
result = pd.concat([test_id, expected_df], axis=1)
result.columns = ['ID','Expected']
result.to_csv(r'submission.csv', index = False)

expected = model.predict(bow2)

In [13]:
#expected = model.predict(bow2)
#test_id = test_data['ID'] # id of item
#expected_df = pd.DataFrame(expected)
#result = pd.concat([test_id, expected_df], axis=1)
#result.columns = ['ID','Expected']
#result.to_csv(r'submission.csv', index = False)